In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import csv
import math
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re

directory = './job_descriptions'
title_directory = './job_titles'
upwork_descriptions = []
upwork_titles = []
no_job_categories = 12

for f in os.listdir(directory):
    print f
    content = open(directory + '/' + f).read()
    upwork_descriptions.append(content)

for f in os.listdir(title_directory):
    print f
    content = open(title_directory + '/' + f).read()
    upwork_titles.append(content)
    
# Extract descriptions from SOC job description list
df = pd.read_csv('./soc_2010_definitions.csv')

soc_descriptions = []
soc_titles = []

for index, row in df.iterrows():
    soc_descriptions.append(row['SOC Definition'])
    soc_titles.append(row['SOC Title'])

complete_job_array = upwork_descriptions + soc_descriptions
complete_title_array = upwork_descriptions + soc_titles
title_job_array = upwork_titles + soc_descriptions
title_title_array = upwork_titles + soc_titles

tfidf = TfidfVectorizer().fit_transform(complete_job_array)
tfidf_title = TfidfVectorizer().fit_transform(complete_title_array)
tfidf_title_job = TfidfVectorizer().fit_transform(title_job_array)
tfidf_title2 = TfidfVectorizer().fit_transform(title_title_array)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = (tfidf * tfidf.T).A
pairwise_title_similarity = (tfidf_title * tfidf_title.T).A
pairwise_title_job_similarity = (tfidf_title_job * tfidf_title_job.T).A
pairwise_title2_similarity = (tfidf_title2 * tfidf_title2.T).A

# Predict job category

def return_job_category(soc_index): 
    print df.iloc[soc_index]['SOC Title']
    
def match_job_category(job_index):
    max_jobs = 5 # Set number of job categories you want to return 
    max_indices = []
    job_array = pairwise_similarity[job_index]
    title_array = pairwise_title_similarity[job_index]
    title_job_array = pairwise_title_job_similarity[job_index]
    title2_array = pairwise_title2_similarity[job_index]
    
    # Find the highest TFIDF value among the job categories we are trying to match to
    soc_jobs = job_array[no_job_categories:]
    soc_titles = title_array[no_job_categories:]
    soc_title_job = title_job_array[no_job_categories:]
    soc_title2 = title2_array[no_job_categories:]
    df['match_' + str(job_index)] = soc_jobs
    df['title_match_' + str(job_index)] = soc_titles
    df['title_job_match_' + str(job_index)] = soc_title_job
    df['title2_match_' + str(job_index)] = soc_title2

    print "----MATCHES ON JOB DESCRIPTION----"
    for i in df.nlargest(5, 'match_' + str(job_index)).index:
        print df[df.index == i]['SOC Title'].values[0]
        print df[df.index == i]['SOC Definition'].values[0]
    
    print "----MATCHES ON JOB TITLES----"
    for i in df.nlargest(5, 'title_match_' + str(job_index)).index:
        print df[df.index == i]['SOC Title'].values[0]
        print df[df.index == i]['SOC Definition'].values[0]
        
    print "----MATCH FROM UPWORK TITLES TO SOC JOBS----"
    for i in df.nlargest(5, 'title_job_match_' + str(job_index)).index:
        print df[df.index == i]['SOC Title'].values[0]
        print df[df.index == i]['SOC Definition'].values[0]
    
    print "----MATCH FROM UPWORK TITLES TO SOC TITLES----"
    for i in df.nlargest(5, 'title2_match_' + str(job_index)).index:
        print df[df.index == i]['SOC Title'].values[0]
        print df[df.index == i]['SOC Definition'].values[0]
        



def match_on_keywords(job_index, soc_dataset):
    
    title_keywords = upwork_titles[job_index]
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(title_keywords)
    #filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            if w != '&':
                filtered_sentence.append(w)
    
    pattern = ""
    for keyword in filtered_sentence:
        pattern += keyword.lower() + " |"
    
    pattern = pattern[:len(pattern)-1]
    print pattern
        
    for job in soc_dataset:
        if re.search(pattern, job.lower()):
            print job

nltk.download('popular')
match_on_keywords(1, soc_titles)
 
#match_job_category(8)

desc_writing.txt
desc_web_mobile_software_dev.txt
desc_translation.txt
desc_sales_and_marketing.txt
desc_legal.txt
desc_it_and_networking.txt
desc_engineering_and_architecture.txt
desc_design_and_creative.txt
desc_data_science.txt
desc_customer_service.txt
desc_administrative_support.txt
desc_accounting_and_consulting.txt
title_writing.txt
title_web_software_dev.txt
title_translation.txt
title_sales_and_marketing.txt
title_legal.txt
title_it_networking.txt
title_engineering_and_architecture.txt
title_design_and_creative.txt
title_data_science.txt
title_customer_service.txt
title_admin_support.txt
title_accounting_and_consulting.txt
[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/ecf384/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/ecf384/nltk_data...
[nltk_data]    |   Package gazetteers is alre

In [33]:
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.similarity(token2))

NameError: name 'nlp' is not defined